In [144]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
import datetime
%load_ext tensorboard
from google.cloud import datastore
import os
import numpy as np
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/home/alice/Downloads/HCL-customer-application-c05a56fa9ccc.json"

client = datastore.Client()


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [145]:
query = client.query(kind="tododata")
x = []
y = []
for i in query.fetch():
    x.append([i['subject'], i['title']])
    y.append([i['totalcoins']])
y = np.asarray(y)

In [146]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False)

In [147]:
vectorizer.fit([i[1] for i in x])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=None, min_df=0,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [148]:
titles = vectorizer.transform([i[1] for i in x]).toarray()

In [149]:
from sklearn import preprocessing

In [150]:
le = preprocessing.LabelEncoder()

In [151]:
le.fit([i[0] for i in x])

LabelEncoder()

In [152]:
subjects = le.transform([i[0] for i in x])
subjects = subjects.reshape(-1,1)


In [153]:
x = np.concatenate((titles,subjects),axis=1)

In [154]:
print(x.shape)
print(y.shape)

(8, 12)
(8, 1)


In [162]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(12, activation='relu',input_shape=(x.shape[1],)))
# Add an output layer with 10 output units:
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [163]:
def Normalize(data, mean_data =None, std_data =None):
    if not mean_data:
        mean_data = np.mean(data)
    if not std_data:
        std_data = np.std(data)
    norm_data = (data-mean_data)/std_data
    return norm_data, mean_data, std_data

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


Y_train, ymean_data, ystd_data = Normalize(y_train)
Y_test,_,_ = Normalize(y_test, ymean_data, ystd_data)

In [168]:
log_dir = "rewardlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train, Y_train, 
          validation_data=(X_test,Y_test), 
          batch_size=20, 
          epochs=30,
          verbose=1,
         callbacks=[tensorboard_callback])

Epoch 1/30
1/1 [==============================] - 0s 113ms/step - loss: 0.0041 - mae: 0.0412 - val_loss: 8.5543 - val_mae: 2.9083
Epoch 2/30
1/1 [==============================] - 0s 112ms/step - loss: 0.0025 - mae: 0.0329 - val_loss: 8.5678 - val_mae: 2.9104
Epoch 3/30
1/1 [==============================] - 0s 122ms/step - loss: 0.0013 - mae: 0.0246 - val_loss: 8.5835 - val_mae: 2.9129
Epoch 4/30
1/1 [==============================] - 0s 111ms/step - loss: 4.7315e-04 - mae: 0.0164 - val_loss: 8.5999 - val_mae: 2.9155
Epoch 5/30
1/1 [==============================] - 0s 132ms/step - loss: 7.9284e-05 - mae: 0.0086 - val_loss: 8.6165 - val_mae: 2.9182
Epoch 6/30
1/1 [==============================] - 0s 140ms/step - loss: 7.5043e-05 - mae: 0.0072 - val_loss: 8.6347 - val_mae: 2.9212
Epoch 7/30
1/1 [==============================] - 0s 144ms/step - loss: 3.6511e-04 - mae: 0.0102 - val_loss: 8.6488 - val_mae: 2.9236
Epoch 8/30
1/1 [==============================] - 0s 133ms/step - loss: 8.

In [169]:
%tensorboard --logdir rewardlogs/fit


Reusing TensorBoard on port 6007 (pid 40440), started 0:02:28 ago. (Use '!kill 40440' to kill it.)

[['dev', 'bayesian optimization'], ['dev', 'GCP MOre data'], ['m340l', 'pass fail m340l'], ['dev', 'meow UI updates'], ['dev', 'meow UI updates'], ['dev', 'meow UI updates'], ['dev', 'meow UI updates'], ['dev', 'meow UI updates']]
